In [ ]:
import os
arch = os.getenv("ARGS", "real")

In [ ]:
# PYTEST_XFAIL_AND_SKIP_NEXT: ROL and fireshape are only expected to work with real numbers
assert arch == "real"

In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import firedrake
else:
    try:
        import firedrake
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
        import firedrake  # noqa: F401

In [ ]:
try:
    import fireshape
except ImportError:
    !python3 -m pip install --no-dependencies git+https://github.com/fireshape/fireshape.git
    import fireshape  # noqa: F401

In [ ]:
%load_ext wurlitzer

In [ ]:
import firedrake as fd
import fireshape as fs
import ROL

In [ ]:
class LevelsetFunctional(fs.ShapeObjective):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # physical mesh
        self.mesh_m = self.Q.mesh_m

        # global function defined in terms of physical coordinates
        x, y = fd.SpatialCoordinate(self.mesh_m)
        self.f = (x - 0.5)**2 + (y - 0.5)**2 - 0.5

    def value_form(self):
        # volume integral
        return self.f * fd.dx

In [ ]:
# setup problem
mesh = fd.UnitSquareMesh(30, 30)
Q = fs.FeControlSpace(mesh)
inner = fs.LaplaceInnerProduct(Q)
q = fs.ControlVector(Q, inner)

In [ ]:
# create objective functional
J = LevelsetFunctional(Q)

In [ ]:
# ROL parameters
params_dict = {
    "Step": {
        "Type": "Line Search",
        "Line Search": {
            "Descent Method": {
                "Type": "Quasi-Newton Step"
            }
        }
    },
    "General": {
        "Secant": {
            "Type": "Limited-Memory BFGS",
            "Maximum Storage": 25
        }
    },
    "Status Test": {
        "Gradient Tolerance": 1e-4,
        "Step Tolerance": 1e-10,
        "Iteration Limit": 30
    }
}

In [ ]:
params = ROL.ParameterList(params_dict, "Parameters")
problem = ROL.OptimizationProblem(J, q)
solver = ROL.OptimizationSolver(problem, params)
solver.solve()